In [1]:
import sys
import pandas as pd
import numpy as np


In [2]:
sys.path.append("../")


In [3]:
import biobss

In [4]:
import neurokit2 as nk
sample_data=pd.read_pickle("..\sample_data\\EDA_Chest.pkl")
sample_data=sample_data.flatten()

In [5]:
decompose=biobss.pipeline.Bio_Process(process_method=biobss.edatools.eda_decompose,modality="EDA",sigtype="EDA",method="highpass")
normalize=biobss.pipeline.Bio_Process(process_method=biobss.signaltools.normalize_signal,modality="EDA",sigtype="EDA")
resample=biobss.pipeline.Bio_Process(process_method=biobss.signaltools.resample_signal_object,modality="EDA",sigtype="EDA",target_sample_rate=350)

In [6]:
pipe=biobss.pipeline.Bio_Pipeline([],modality="EDA",sigtype="EDA",windowed_process=True,window_size=60,step_size=30)

In [7]:
pipe.preprocess_queue.add_process(normalize)
pipe.preprocess_queue.add_process(resample)
pipe.process_queue.add_process(decompose)

In [8]:
pipe.set_input(sample_data,700,"EDA","EDA")

In [9]:
pipe

Bio_Pipeline:
	Modality: EDA
	Signal Type: EDA
	Preprocessors: <biobss.pipeline.process_list.Process_List object at 0x000001D392A09AF0>
	Processors: <biobss.pipeline.process_list.Process_List object at 0x000001D392A09A90>
	Postprocessors: <biobss.pipeline.process_list.Process_List object at 0x000001D392A09D00>
	Window Size(Seconds): 60
	Step Size: 30

In [10]:
pipe.run_pipeline()

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
pipe.processed_signal

EDA Signal: EDA_Raw
Sampling Rate: 700
Channels: 3
Channel Names: ['EDA_Raw', 'EDA_Tonic', 'EDA_Phasic']
Multichannel: True
Signal: [array([-1.34650432, -1.33614792, -1.34509208, ...,  0.95167532,
        0.96014874,  0.95826576]), array([-1.34382019, -1.34381905, -1.3438179 , ...,  0.96029843,
        0.96029843,  0.96029843]), array([-0.00268412,  0.00767113, -0.00127418, ..., -0.01124827,
       -0.00277935, -0.00466684])]

In [ ]:
bio_data=biobss.pipeline.Bio_Data()

In [ ]:
eda_channel=biobss.pipeline.Data_Channel(sample_data,sampling_rate=700,name="EDA_Raw",modality="EDA")

In [ ]:
eda_channel

EDA_Raw (EDA) (700Hz) (5920.0s) ((4144000,)) ((4144000,)) (float64)[5.40046692 5.40885925 5.40161133 ... 7.26280212 7.26966858 7.2681427 ]

In [ ]:
bio_data.add_channel(eda_channel)

In [ ]:
bio_data

Signal object with 1 channel(s)EDA_Raw (EDA) (700Hz) (5920.0s) ((4144000,))